In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, IntRangeSlider, interactive
from IPython.display import display, clear_output
from commpy.channels import awgn  # Import AWGN (Additive White Gaussian Noise) function from commpy
import time

# Parameters
bits_per_symbol = 4  # Number of bits per symbol for M-FSK
num_symbols = 1000  # Number of symbols to simulate
num_samples = 80  # Number of samples per symbol (oversampling factor)

# Derived parameters
M = 2 ** bits_per_symbol  # Number of different symbols (M-FSK)
baud_rate = 1  # Baud rate (symbol rate)
carrier_freq = 2 * M * baud_rate  # Carrier frequency for FSK signal

# Calculate total number of bits and symbol/sampling periods
num_bits = bits_per_symbol * num_symbols  # Total number of bits
symbol_period = 1 / baud_rate  # Symbol period
sampling_period = symbol_period / num_samples  # Sampling period

# M frequencies spaced within the coherent distance (Baud Rate)
frequencies = carrier_freq + (baud_rate / 2) * (np.arange(1, M + 1) - (M + 1) / 2)

# Find the maximum frequency
max_frequency = np.max(frequencies)

# Recalculate the number of samples to ensure the sampling frequency (Fs) meets Nyquist criteria
sampling_freq = 2 * max_frequency
num_samples = int(np.ceil(sampling_freq / baud_rate)) + 10  # Adjust number of samples

# Recalculate the sampling period based on updated num_samples
sampling_period = symbol_period / num_samples

# Generate random input bits and reshape them into symbols
bits = np.random.randint(0, 2, num_bits)
bit_matrix = bits.reshape((num_symbols, bits_per_symbol))

# Time vectors
time_vector = np.arange(0, len(bit_matrix) * symbol_period, symbol_period)  # Time vector for symbols
oversampling_time_vector = np.arange(0, symbol_period, sampling_period)  # Time vector for oversampling

# Generate the FSK signal
fsk_signal = []
amplitude = np.sqrt(2 / symbol_period / num_samples)  # Normalize amplitude
for symbol_index in range(len(bit_matrix)):
    # Convert binary symbol to frequency index
    freq_symbol = frequencies[int(''.join(map(str, bit_matrix[symbol_index])), 2)]
    time_segment = (symbol_index * symbol_period) + oversampling_time_vector
    fsk_signal.append(np.sin(2 * np.pi * freq_symbol * time_segment))  # FSK modulation
fsk_signal = np.concatenate(fsk_signal)  # Concatenate the signal

# Function to calculate number of errors for a range of Eb/No values
def calculate_errors(EbNo_range):
    clear_output(wait=True)  # Clear previous output before updating
    EbNo_values = list(range(EbNo_range[0], EbNo_range[1] + 1))  # Range of Eb/No values
    errors_list = []  # List to store the number of errors for each Eb/No

    for EbNo in EbNo_values:
        # Calculate SNR based on Eb/No, bits per symbol, and oversampling
        SNR = EbNo + 10 * np.log10(bits_per_symbol) - 10 * np.log10(num_samples / 2)  # SNR in dB

        # Add AWGN to the FSK signal
        noisy_signal = awgn(fsk_signal, SNR)

        # FSK receiver (demodulation and decoding)
        received_symbols = []
        for symbol_index in range(len(noisy_signal) // num_samples):
            time_segment = (symbol_index * symbol_period) + oversampling_time_vector
            signal_segment = noisy_signal[symbol_index * num_samples:(symbol_index + 1) * num_samples]

            # Correlate with each possible frequency to find the best match
            match_scores = []
            for freq in frequencies:
                signal_template = np.sin(2 * np.pi * freq * time_segment)
                match_scores.append(np.sum(signal_segment * signal_template))
            
            decoded_symbol = np.argmax(match_scores)  # Find the symbol with the highest match score
            received_symbols.append([int(bit) for bit in bin(decoded_symbol)[2:].zfill(bits_per_symbol)])  # Decode symbol
        received_symbols = np.array(received_symbols).reshape((num_symbols, bits_per_symbol))

        # Count bit errors by comparing transmitted and received symbols
        errors = np.sum(bit_matrix != received_symbols)
        errors_list.append(errors)

    # Plot the number of errors against Eb/No
    plt.figure(figsize=(10, 6))
    plt.plot(EbNo_values, errors_list, marker='o', linestyle='-', markersize=8)
    plt.xlabel('Eb/No (dB)')
    plt.ylabel('Number of Errors')
    plt.title('Number of Errors vs. Eb/No')
    plt.grid(True)
    plt.show()

# Create a slider widget for Eb/No range
EbNo_slider = IntRangeSlider(
    value=[0, 20],  # Initial range
    min=0,
    max=20,
    step=1,
    description='EbNo (dB):',
    continuous_update=False,  # Update only on release of slider
    layout=Layout(width='99%')  # Slider layout
)

# Create an interactive widget to run the simulation when the slider changes
interactive_plot = interactive(calculate_errors, EbNo_range=EbNo_slider)

# Combine the slider and plot into a VBox layout
input_widgets = VBox([EbNo_slider], layout=Layout(flex='1 1 auto', width='auto'))
plot_output = interactive_plot.children[-1]  # Get the plot output

# Create a VBox layout containing both input widgets and plot output
ui = VBox([input_widgets, plot_output])

# Display the UI components
clear_output(wait=True)  # Clear previous output if necessary
display(ui)
